# Building an Anime Recommender System

In [5]:
import sys
sys.path.append("..")

from animerec import models
from animerec import utils

import pandas
import pickle

### 0. Train Model

In [6]:
import os
import sys
import time
import pickle


def get_script_path():
    return os.path.dirname(os.path.realpath(sys.argv[0]))

sys.path.append(get_script_path()+"/..")


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from animerec.models import MatrixFactorization as MF


param_grid = {
    'n_features': [5, 10, 25, 50, 100, 150, 200],
    'reg': [0.1, 1, 10, 20, 50]
}


# Load Dataset
pickle_in = open("../data/scores.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()


Xtrain, Xtest = train_test_split(X, test_size=0.1)
grid_search = GridSearchCV(MF(), param_grid, cv=5, n_jobs=-1, verbose=3)

t0 = time.time()
grid_search.fit(Xtrain)

print("done in %0.3fs" % (time.time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))+ numpy.eye(model.n_features)*model.reg


Fitting 5 folds for each of 35 candidates, totalling 175 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


TypeError: fit() got an unexpected keyword argument 'verbose'

In [4]:
grid_search

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=<class 'animerec.models.MatrixFactorization'>,
             iid='warn', n_jobs=-1,
             param_grid={'n_features': [5, 10, 25, 50, 100, 150, 200],
                         'reg': [0.1, 1, 10, 20, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

### 1. Load Dataset and Model

In [ ]:
# Load anime metadata.
pickle_in = open("../data/animes.pickle", 'rb')
animes = pickle.load(pickle_in)
animes.index = animes['anime_id']
animes = animes[ ['title', 'title_english', 'type', 'premiered', 'genre'] ]
pickle_in.close()

# Load model trained on server.
pickle_in = open("../model/model.pickle", 'rb')
model = pickle.load(pickle_in)
pickle_in.close()

# Load anime id's.
pickle_in = open("../model/item-id.pickle", 'rb')
item_id = pickle.load(pickle_in)
pickle_in.close()

### 2. Webscrape current profile and create prediction

In [ ]:
myprofile = utils.get_user_anime_list('Manuel')
x = utils.get_score_vector_from_user_anime_list(myprofile, item_id)

In [ ]:
rhat = model.predict(x)

In [ ]:
prediction = utils.prediction_to_dataframe(rhat, myprofile, item_id, keep_all=False)
prediction = pandas.DataFrame(prediction)
prediction.columns = ['score']
prediction = prediction.join(animes)

In [ ]:
for i,a in prediction.sort_values(by='score', ascending=False).head(20).iterrows():
    print(f"{a.score:.2f} - {a.title} ({a.premiered})")

In [ ]:
filt = prediction.apply(lambda x: "Sci-Fi" in str(x.genre), axis=1)
prediction[filt].sort_values(by='score', ascending=False).head(10)

In [ ]:
filt = prediction.apply(lambda x: "Fantasy" in str(x.genre), axis=1)
prediction[filt].sort_values(by='score', ascending=False).head(10)

In [ ]:
filt = prediction.apply(lambda x: "Slice of Life" in str(x.genre), axis=1)
prediction[filt].sort_values(by='score', ascending=False).head(10)